**<font  size=6>
使用Excel表格讀取並合併
</font>**

In [1]:
import pandas as pd

In [2]:
def make_melt_table(table_path,feature_names):
    #讀取數據
    df2000 = pd.read_excel(table_path,index_col=0).reset_index(drop =True).iloc[:,3:].reset_index(drop =True)
    df2000['日期'] = df2000['股票代號'].str[0:8]
    stock_name_table = pd.DataFrame({'股票代號':df2000.columns[1:-2],'股票名稱':df2000.iloc[0,1:-2]}).reset_index(drop = True)
    df2000 = df2000.iloc[1:,:]
    del df2000['股票代號']
    df2000 = df2000.set_index(['日期']).reset_index(drop = False)
    df2000_melt = pd.melt(df2000, id_vars= '日期', value_vars=list(df2000.columns[1:]))
    df2000_melt = df2000_melt.rename(columns={'variable':'股票代號','value':feature_names})
    df2000_melt = df2000_melt.merge(stock_name_table,on='股票代號')
    return df2000_melt


In [3]:
Volume2000 = make_melt_table("Z:\\07個人\\ChiehLin\\數據\\2000數據\\成交量2000.xlsx",'成交金額(千)')
close_price_2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\收盤價2000.xlsx','收盤價')
Lowest_price2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\最低價2000.xlsx','最低價')
Highest_price2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\最高價2000.xlsx','最高價')
open_price2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\開盤價2000.xlsx','開盤價')
Financing_used2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\融資使用率2000.xlsx','融資使用率')
Financing_retent2000 =  make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\融資維持率2000.xlsx','融資維持率')
restored_close_price2000 =  make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\還原收盤2000.xlsx','還原收盤價')

In [4]:
df_melt = Volume2000.set_index('股票名稱').reset_index(drop = False)
df_melt = df_melt.merge(close_price_2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(Lowest_price2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(Highest_price2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(open_price2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(Financing_used2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(Financing_retent2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(restored_close_price2000, on= ['日期', '股票代號', '股票名稱'])

In [5]:
df_melt

,股票名稱,日期,股票代號,成交金額(千),收盤價,最低價,最高價,開盤價,融資使用率,融資維持率,還原收盤價
0,台泥,20140506,1101,279876,46.8,46.65,47.05,46.85,0.36,164.1368,24.58
1,台泥,20140507,1101,687831,46,46,46.65,46.6,0.41,162.0568,24.16
2,台泥,20140508,1101,573230,45.7,45.5,46.2,46.1,0.49,161.901,24
3,台泥,20140509,1101,599689,45.3,44.75,45.7,45.5,0.51,161.0522,23.79
4,台泥,20140512,1101,606761,44.2,44,45,44.9,0.54,158.1334,23.21
...,...,...,...,...,...,...,...,...,...,...,...
3534911,邁達康,20220622,9960,363,26,25.8,26,25.8,13.3,182.8924,26
3534912,邁達康,20220623,9960,0,25.95,25.95,25.95,25.95,13.3,182.5407,25.95
3534913,邁達康,20220624,9960,441,26,25.3,26.05,25.7,13.24,182.9065,26
3534914,邁達康,20220627,9960,338,26.35,25.7,26.35,25.8,13.24,185.3687,26.35


**<font  size=6>
不須導出Excel表格，直接讀取Cmoney mysql數據庫數據到python內存方法介紹
</font>**

**<font  size=5 color = "red">注意:要先在python運行環境中安裝pyodbc包，方法與安裝pandas等包相同。</font>**

以讀取20140506以後的台泥的融資使用率與融資維持率為例

In [2]:
import pyodbc
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

連結到Cmoney數據庫

In [3]:
DB_Host = "192.168.1.16"
DB_Name = "X01"
DB_User = "cm23357868"#以我的帳號為例，請填入您個人帳號
DB_Password = "86875332"#以我的密碼為例，請填入您個人密碼

print("Connecting to database using pyodbc...")

db = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; SERVER=' + DB_Host + '; DATABASE=' + DB_Name + '; UID=' + DB_User + '; PWD=' + DB_Password)

print("Succesfully Connected to database using pyodbc!")

cursor = db.cursor()

Connecting to database using pyodbc...
Succesfully Connected to database using pyodbc!


找到融資使用率與融資維持率的表格路徑為[X01].[dbo].[日融資券排行]

In [4]:
query = """SELECT  [日期]
      ,[股票代號]
      ,[股票名稱]
      ,[資使用率]
      ,[融資維持率(%)]
  FROM [X01].[dbo].[日融資券排行] WITH(NOLOCK) where 股票代號 = '1101' And 日期 >= 20140506 order by 日期 """  

#FROM後填入表格路徑  WHERE指令可以選定只讀取部分數據的條件 order by 可按照特定行排序 若要使用降序 可以寫成 order by 日期 desc
#延伸閱讀可Google "Sql語法" ，也可以問我

df = pd.read_sql(query, db)

In [5]:
df

,日期,股票代號,股票名稱,資使用率,融資維持率(%)
0,20140506,1101,台泥,0.36,164.136795
1,20140507,1101,台泥,0.41,162.056793
2,20140508,1101,台泥,0.49,161.901001
3,20140509,1101,台泥,0.51,161.052200
4,20140512,1101,台泥,0.54,158.133408
...,...,...,...,...,...
2009,20220718,1101,台泥,0.52,161.517899
2010,20220719,1101,台泥,0.52,165.367493
2011,20220720,1101,台泥,0.55,165.358307
2012,20220721,1101,台泥,0.55,152.717499
